In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, BatchNormalization, GlobalAveragePooling1D
from sklearn.metrics import classification_report
from scipy.stats import skew, kurtosis
import requests

In [2]:
API_BASE_URL = "https://bills-api.parliament.uk/api/v1/Bills"

In [3]:

def get_bill_info(bill_id):
    url = f"{API_BASE_URL}/{bill_id}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    try:
        data = response.json()
        if isinstance (data, dict):
            long_title = data.get("longTitle", "N/A")
            short_title = data.get("shortTitle", "N/A")
            orig_house = data.get("originatingHouse", "N/A")
            status = check_bill_progress(bill_id)
            sessionID = data.get("introducedSessionId", "N/A")
           
            
            return bill_id, long_title, short_title, orig_house, status, sessionID
    except requests.exceptions.JSONDecodeError:
        print(f"Error: Response for Bill ID {bill_id} is not valid JSON. Response:", response.text)
        return False
    
    

        




In [4]:

def count_bills_in_session(session_id):
    """Fetch the number of bills that contain a specific session ID."""
    params = {
        "Session": session_id,
        "Take": 1  # Only request one item to check total count
    }
    
    headers = {"accept": "application/json"}
    response = requests.get(API_BASE_URL, headers=headers, params=params)

    try:
        data = response.json()
        if isinstance(data, dict) and "totalResults" in data:
            return data["totalResults"]  # Total number of bills with this session ID
        else:
            print("Unexpected API response format:", data)
            return 0
    except requests.exceptions.JSONDecodeError:
        print("Error: Response is not valid JSON. Response:", response.text)
        return 0
    
print (count_bills_in_session(37))

355


In [58]:
def check_bill_progress(bill_id):
    """Checks if a bill is an act using the isAct field from the API."""
    url = f"https://bills-api.parliament.uk/api/v1/Bills/{bill_id}"  
    headers = {"accept": "application/json"}  
    
    response = requests.get(url, headers=headers)

    try:
        data = response.json()  
        if 'isAct' in data:
            return 2 if data['isAct'] else 1
        else:
            print(f"'isAct' field not found for Bill ID {bill_id}")
            return 3  

    except requests.exceptions.JSONDecodeError:
        print(f"Error: Response for Bill ID {bill_id} is not valid JSON. Response:", response.text)
        return False
print (
check_bill_progress_2(799))

2


In [59]:
def populate_dataframes(bills, f_commons, f_lords, s_commons, s_lords, weird):
    columns1 = ["Bill Id", "Long Title", "Short Title", "Origin_House", "Status", "SessionID"]

    rows_f_commons, rows_f_lords, rows_s_commons, rows_s_lords, rows_weird = [], [], [], [], []
    
    for bill in bills:
        bill_id = bill.get("billId", "N/A")
        bill_id, long_title, short_title, orig_house, status, sessionID = get_bill_info(bill_id)
        
        new_row = [bill_id, long_title, short_title, orig_house, status, sessionID]
        
        if status == 1:
            if orig_house == "Commons":
                rows_f_commons.append(new_row)
            elif orig_house == "Lords":
                rows_f_lords.append(new_row)
        elif status == 2:
            if orig_house == "Commons":
                rows_s_commons.append(new_row)
            elif orig_house == "Lords":
                rows_s_lords.append(new_row)
        elif status == 3:
            rows_weird.append(new_row)
    

    f_commons = pd.concat([pd.DataFrame(rows_f_commons, columns=columns1), f_commons], ignore_index=True)
    f_lords = pd.concat([pd.DataFrame(rows_f_lords, columns=columns1), f_lords], ignore_index=True)
    s_commons = pd.concat([pd.DataFrame(rows_s_commons, columns=columns1), s_commons], ignore_index=True)
    s_lords = pd.concat([pd.DataFrame(rows_s_lords, columns=columns1), s_lords], ignore_index=True)
    weird = pd.concat([pd.DataFrame(rows_weird, columns=columns1), weird], ignore_index=True)
    
    return f_commons, f_lords, s_commons, s_lords, weird


In [67]:
import datetime




#last session 39
columns = ["Bill Id", "Long Title", "Short Title", "Origin_House", "Status", "SessionID"]

S_commons0 = pd.DataFrame(columns=columns)
F_commons0 = pd.DataFrame(columns=columns)
S_lords0 = pd.DataFrame(columns=columns)
F_lords0 = pd.DataFrame(columns=columns)
weirdList0 = pd.DataFrame(columns=columns)



def get_bills_by_session(session_id, num_bills):
    """Fetch bills from a specific session ID."""
    params = {
        "Session": session_id,
        "Take": num_bills  
    }
    
    headers = {"accept": "application/json"}
    response = requests.get(API_BASE_URL, headers=headers, params=params)

    try:
        data = response.json()
        if isinstance(data, dict) and "items" in data:
            return data["items"]  
        else:
            print("Unexpected API response format:", data)
            return []
    except requests.exceptions.JSONDecodeError:
        print("Error: Response is not valid JSON. Response:", response.text)
        return []
    



session_id = int(input("Enter Session ID: "))
#num_bills = int(input("Enter number of bills to retrieve: "))






start_session = session_id
end_session = 39 

#num_bills0= count_bills_in_session(session_id)
#bills0= get_bills_by_session(session_id, num_bills)


now = datetime.datetime.now()

# Print just the time part
print("Code starts executing at :", now.strftime("%H:%M:%S"))  
    

    
    
for session in range(start_session, end_session + 1):
    now = datetime.datetime.now()
    print(f"counting bills in session {session} starts at :", now.strftime("%H:%M:%S"))  
    num_bills = count_bills_in_session(session)
    now = datetime.datetime.now()
    print(f"counting bills in session {session} finish at :", now.strftime("%H:%M:%S"))  
    bills0 = get_bills_by_session(session, num_bills)
    now = datetime.datetime.now()
    print(f"Bills0 computed for session {session} finish at :", now.strftime("%H:%M:%S"))  
    
    F_commons0 , F_lords0, S_commons0, S_lords0, weirdList0 = populate_dataframes(bills0, F_commons0, F_lords0, S_commons0, S_lords0, weirdList0)    
    
    now = datetime.datetime.now()
    print(f"Dataframes appended at {session} finish at :", now.strftime("%H:%M:%S"))
    



display(F_commons0)
display(F_lords0)
display(S_commons0)
display(S_lords0)


#print ("This is the weird list")
#display(weirdList0)

F_commons0.to_csv(   f"18_{start_session} to {end_session}.csv"   , index=False)
F_lords0.to_csv(     f"19_{start_session} to {end_session}.csv"   , index = False )
S_lords0.to_csv(     f"29_{start_session} to {end_session}.csv", index = False )
S_commons0.to_csv(   f"28_{start_session} to {end_session}.csv", index= False)
weirdList0.to_csv(   f"errors {start_session} to {end_session}.csv", index= False )



print("Code finished executing ")
# Get the current datetime
now2 = datetime.datetime.now()

# Print just the time part
print("Time Code Finished Executing:", now2.strftime("%H:%M:%S"))




Enter Session ID: 17
Code starts executing at : 01:47:36
counting bills in session 17 starts at : 01:47:36
counting bills in session 17 finish at : 01:47:38
Bills0 computed for session 17 finish at : 01:47:38
Dataframes appended at 17 finish at : 01:47:38
counting bills in session 18 starts at : 01:47:38
counting bills in session 18 finish at : 01:47:40
Bills0 computed for session 18 finish at : 01:47:42
Dataframes appended at 18 finish at : 01:47:42
counting bills in session 19 starts at : 01:47:42
counting bills in session 19 finish at : 01:47:43
Bills0 computed for session 19 finish at : 01:47:44
Dataframes appended at 19 finish at : 01:47:47
counting bills in session 20 starts at : 01:47:47
counting bills in session 20 finish at : 01:47:47
Bills0 computed for session 20 finish at : 01:47:49
Dataframes appended at 20 finish at : 01:49:51
counting bills in session 21 starts at : 01:49:51
counting bills in session 21 finish at : 01:49:52
Bills0 computed for session 21 finish at : 01:4

,Bill Id,Long Title,Short Title,Origin_House,Status,SessionID
0,3785,A Bill to make provision about absent voting i...,Absent Voting (Elections in Scotland and Wales...,Commons,1,39
1,3800,A Bill to require providers of electronic comm...,Access to Telecommunications Networks Bill,Commons,1,39
2,3814,A Bill to prohibit the granting of planning pe...,Agricultural Land (Planning) Bill,Commons,1,39
3,3790,A Bill to make provision for and in connection...,"Animal Welfare (Import of Dogs, Cats and Ferre...",Commons,1,39
4,3798,A Bill to require a person in charge of a dog ...,Animal Welfare (Responsibility for Dog on Dog ...,Commons,1,39
...,...,...,...,...,...,...
2411,106,None,Town and Country Planning (Amendment) Bill,Commons,1,20
2412,107,Make provision for the law relating to the rig...,Trade Union Rights and Freedoms Bill,Commons,1,20
2413,129,Require the Secretary of State to lay before t...,Waging War (Parliament's Role and Responsibilt...,Commons,1,20
2414,141,None,Wild Birds (Protection) Bill,Commons,1,20


,Bill Id,Long Title,Short Title,Origin_House,Status,SessionID
0,3942,A bill to make provision for the regulation of...,Artificial Intelligence (Regulation) Bill [HL],Lords,1,39
1,3741,"A Bill to allow adults who are terminally ill,...",Assisted Dying for Terminally Ill Adults Bill ...,Lords,1,39
2,3758,A Bill to introduce a minimum period of 56 day...,Asylum Support (Prescribed Period) Bill [HL],Lords,1,39
3,3734,A Bill to make provision about recapitalisatio...,Bank Resolution (Recapitalisation) Bill [HL],Lords,1,39
4,3910,A bill to make provision about local and schoo...,Bus Services (No. 2) Bill [HL],Lords,1,39
...,...,...,...,...,...,...
677,63,Make provision for the establishment of a Roya...,Royal Commission (Slavery) Bill [HL],Lords,1,20
678,100,Make provision about the advertising of food p...,Television Advertising (Food) Bill [HL],Lords,1,20
679,105,Make provision for actions for damages for tor...,Torture (Damages) Bill [HL],Lords,1,20
680,621,A Bill to confer further powers upon Transport...,Transport for London (Supplemental Toll Provis...,Lords,1,20


,Bill Id,Long Title,Short Title,Origin_House,Status,SessionID
0,3731,A Bill to impose duties on the Treasury and th...,Budget Responsibility Act 2024,Commons,2,39
1,3882,A Bill to make provision for loans or other fi...,Financial Assistance to Ukraine Act 2025,Commons,2,39
2,3732,A Bill to make provision for passenger railway...,Passenger Railway Services (Public Ownership) ...,Commons,2,39
3,3736,A Bill to authorise the use of resources for t...,Supply and Appropriation (Main Estimates) Act ...,Commons,2,39
4,3533,A Bill to Prohibit the export of certain lives...,Animal Welfare (Livestock Exports) Act 2024,Commons,2,38
...,...,...,...,...,...,...
554,140,Make provision for a railway transport system ...,Crossrail Act 2008,Commons,2,18
555,208,To confer further powers upon local authoritie...,LONDON LOCAL AUTHORITIES AND TRANSPORT FOR LON...,Commons,2,18
556,144,Make provision about social security; to amend...,Welfare Reform Act 2007,Commons,2,19
557,140,Make provision for a railway transport system ...,Crossrail Act 2008,Commons,2,18


,Bill Id,Long Title,Short Title,Origin_House,Status,SessionID
0,3733,A Bill to amend the Arbitration Act 1996; and ...,Arbitration Act 2025,Lords,2,39
1,3735,"A Bill to make provision about the status of, ...",Commonwealth Parliamentary Association and Int...,Lords,2,39
2,3744,A Bill to extend the period within which vacan...,Lords Spiritual (Women) Act 2015 (Extension) A...,Lords,2,39
3,3751,"A Bill to make provision about the regulation,...",Water (Special Measures) Act 2025,Lords,2,39
4,3506,A Bill to regulate the use of automated vehicl...,Automated Vehicles Act 2024,Lords,2,38
...,...,...,...,...,...,...
175,84,To provide for joint departments of the Houses...,Parliament (Joint Departments) Act 2007,Lords,2,20
176,79,Make provision about serious crime prevention ...,Serious Crime Act 2007,Lords,2,20
177,210,To confer further powers upon Transport for Lo...,Transport for London Act 2008,Lords,2,19
178,109,Make provision about tribunals and inquiries; ...,"Tribunals, Courts and Enforcement Act 2007",Lords,2,20


Code finished executing 
Time Code Finished Executing: 02:30:29


In [64]:

total_bills = 0
yearsWithBills=0

for i in range (39,50):
    bills = count_bills_in_session(i)
    if bills != 0:
        yearsWithBills = yearsWithBills + 1
    print( (f"Bills in session {i} is {bills}"))
    total_bills = total_bills + bills
print(total_bills)
print (f"Years with bills {yearsWithBills}")



Bills in session 39 is 216
Bills in session 40 is 0
Bills in session 41 is 0
Bills in session 42 is 0
Bills in session 43 is 0
Bills in session 44 is 0
Bills in session 45 is 0
Bills in session 46 is 0
Bills in session 47 is 0
Bills in session 48 is 0
Bills in session 49 is 0
216
Years with bills 1
